### LongTR report allele length difference rather than the absolute length

### Generating Truth set from Assembly (requires bed output from AAM.py)

In [ ]:
### 2.9 MILLION LOCI
##
# variable 'value' & '*at_motif_dict[key]' extraction index position vary based on the BED input file provided to the AAM.py
import sys
mat_dict = {}
mat_motif_dict = {}
with open('Maternal_allele.bed') as mh:
    for line in mh:
        line = line.strip().split('\t')
        key = f'{line[0]}:{line[1]}-{line[2]}'
        value = line[-3] # allele length value
        if value.isnumeric():
            mat_dict[key] = int(value) - (int(line[2]) - int(line[1])) # calc length difference wrt reference allele length
            mat_motif_dict[key] = int(float(line[4])) # This has to be motif length as integer
pat_dict = {}
pat_motif_dict = {}
with open('Paternal_allele.bed') as ph:
    for line in ph:
        line = line.strip().split('\t')
        key = f'{line[0]}:{line[1]}-{line[2]}'
        value = line[-3] # allele length value
        if value.isnumeric():
            pat_dict[key] = int(value) - (int(line[2]) - int(line[1])) # calc length difference wrt reference allele length
            pat_motif_dict[key] = int(float(line[4])) # This has to be motif length as integer 
            
print('length of Mat and Pat assembly dict = ', len(mat_dict), len(pat_dict))
print('\n\n')

# Merging allele lengths from the parent into a dict
Assembly_dict = {}
Assembly_motif_dict = {}
for key in mat_dict:
    if key in pat_dict:
        if mat_motif_dict[key] == pat_motif_dict[key]:
            Assembly_dict[key] = {mat_dict[key], pat_dict[key]}
            Assembly_motif_dict[key] = mat_motif_dict[key]
            
print('length of assembly dict = ', len(Assembly_dict))

### Categorising the truth set based on zygosity

In [5]:
Assembly_homozygous_dict = {}
Assembly_heterozygous_dict = {}
nonzygous = 0
for locus_key in Assembly_dict:
    if len(Assembly_dict[locus_key]) == 1:
        Assembly_homozygous_dict[locus_key] = Assembly_dict[locus_key]
    elif len(Assembly_dict[locus_key]) == 2:
        Assembly_heterozygous_dict[locus_key] = Assembly_dict[locus_key]
    else: nonzygous += 1
print('length of Assembly_homozygous_dict = ', len(Assembly_homozygous_dict))
print('length of Assembly_heterozygous_dict = ', len(Assembly_heterozygous_dict))
print('length of nonzygous = ', nonzygous)

length of Assembly_homozygous_dict =  2575955
length of Assembly_heterozygous_dict =  199015
length of nonzygous =  0


### Allele length extraction from LongTR VCFs

In [ ]:
import gzip

N_hifi_dict = {}
with gzip.open("tr_calls.vcf.gz", 'rb') as fh:
    for line in fh:
        line = line.decode('utf-8') # use 'rb'
        if line[0]!="#":
            line = line.strip().split("\t")
            if line[9]=='.': continue
            info_split = line[7].split(";")
            start = info_split[0].split('=')[1] # start
            end = info_split[1].split('=')[1] # end
            key = f'{line[0]}:{start}-{end}'
            alleles = line[9].split(':')[1].split('|')
            allele_set = set()
            for i in alleles:
                allele_set.add(int(i))
            type = 0
            if len(allele_set) == 1:
                zygous = 0
            else: zygous = 1
            value = [allele_set, zygous, type]

            N_hifi_dict[key] = value
print('length of HiFi dict = ', len(N_hifi_dict))

N_sim5_dict = {}
print('length of simplex_5kHz dict = ', len(N_sim5_dict))


N_simHQ_dict = {}
print('length of simplexHQ dict = ', len(N_simHQ_dict))

N_dup_dict = {}
print('length of duplex dict = ', len(N_dup_dict))

N_sim4_dict = {}
print('length of simplex_4kHz dict = ', len(N_sim4_dict))

## Concordance Calculation

In [ ]:
for idx,dicts in enumerate([N_sim4_dict, N_sim5_dict, N_simHQ_dict, N_dup_dict, N_hifi_dict]):
    if len(dicts) == 0: continue
    if idx == 0: print('***Simplex4kHz***')
    elif idx == 1: print('***Simplex5kHz***')
    elif idx == 2: print('***Simplex-HQ***')
    elif idx == 3: print('***Duplex***')
    else: print('***Hifi***')
    common_loci_btw_assembly_and_sim = len(set(list(Assembly_dict.keys())) & set(list(dicts.keys())))
    print('common_loci_btw_assembly_and_sample = ', common_loci_btw_assembly_and_sim)

    common_loci_btw_HOMO_assembly_and_sim = len(set(list(Assembly_homozygous_dict.keys())) & set(list(dicts.keys())))
    print('common_loci_btw_HOMO_assembly_and_sample = ', common_loci_btw_HOMO_assembly_and_sim)

    common_loci_btw_HETERO_assembly_and_sim = len(set(list(Assembly_heterozygous_dict.keys())) & set(list(dicts.keys())))
    print('common_loci_btw_HETERO_assembly_and_sample = ', common_loci_btw_HETERO_assembly_and_sim)

    # both tool & assembly says as Homozygous
    sim_homo_homo_match = 0
    sim_homo_homo_1bp_diff = 0
    sim_homo_homo_40m_diff = 0 # allele length differ by 40% of motif length 
    sim_homo_homo_mismatch = 0 # difference beyond 40%

    # Assembly is Homozygous but tool says Heterozygous
    sim_homo_hetero_match_diff1bp = 0 # one allele is matching another one os differing by 1 bp
    sim_homo_hetero_match_40m_diff = 0
    sim_homo_hetero_match_mismatch = 0
    sim_homo_hetero_both_1bpdiff = 0
    sim_homo_hetero_both_40m_diff = 0
    sim_homo_hetero_1bdfiff_40m_diff = 0
    sim_homo_hetero_1bpdiff_mismatch = 0
    sim_homo_hetero_40m_diff_mismatch = 0
    sim_homo_hetero_both_mismatch = 0


    for locus_key in Assembly_homozygous_dict:
        
        if locus_key in dicts:
            Value = dicts[locus_key]
            motif_thresh = 0.4*Assembly_motif_dict[locus_key]

            if Value[1] == 0: # homozygous
                sim_allele = list(Value[0])[0]
                ass_allele = list(Assembly_homozygous_dict[locus_key])[0]
                if sim_allele == ass_allele:

                    sim_homo_homo_match +=1

                        
                elif abs(sim_allele-ass_allele) == 1:

                    sim_homo_homo_1bp_diff += 1
                elif abs(sim_allele-ass_allele) <= motif_thresh: sim_homo_homo_40m_diff += 1
                else:
                    sim_homo_homo_mismatch += 1

            else:
                sim_allele = Value[0]
                ass_allele = Assembly_homozygous_dict[locus_key]
                if len(sim_allele-ass_allele) == 1:
                    if abs(list(sim_allele-ass_allele)[0] - list(ass_allele)[0]) == 1: 
                        sim_homo_hetero_match_diff1bp += 1
                    elif abs(list(sim_allele-ass_allele)[0] - list(ass_allele)[0]) <= motif_thresh:
                        sim_homo_hetero_match_40m_diff += 1
                    else: 
                        sim_homo_hetero_match_mismatch += 1
                else:
                    if (abs(list(sim_allele)[0] - list(ass_allele)[0]) == 1) and (abs(list(sim_allele)[1] - list(ass_allele)[0]) == 1):
                        sim_homo_hetero_both_1bpdiff += 1
                    elif (abs(list(sim_allele)[0] - list(ass_allele)[0]) == 1) or (abs(list(sim_allele)[1] - list(ass_allele)[0]) == 1):
                        list_sim_allele = list(sim_allele)
                        if abs(list_sim_allele[0] - list(ass_allele)[0]) == 1:
                            if abs(list(sim_allele)[1] - list(ass_allele)[0]) <= motif_thresh:
                                sim_homo_hetero_1bdfiff_40m_diff +=1
                            else:
                                sim_homo_hetero_1bpdiff_mismatch += 1
                        else:
                            if abs(list(sim_allele)[0] - list(ass_allele)[0]) <= motif_thresh:
                                sim_homo_hetero_1bdfiff_40m_diff +=1
                            else:
                                sim_homo_hetero_1bpdiff_mismatch += 1
                    elif (abs(list(sim_allele)[0] - list(ass_allele)[0]) <= motif_thresh) and (abs(list(sim_allele)[1] - list(ass_allele)[0]) <= motif_thresh):
                        sim_homo_hetero_both_40m_diff += 1    
                    elif (abs(list(sim_allele)[0] - list(ass_allele)[0]) <= motif_thresh) or (abs(list(sim_allele)[1] - list(ass_allele)[0]) <= motif_thresh):    
                        sim_homo_hetero_40m_diff_mismatch += 1
                    else: sim_homo_hetero_both_mismatch += 1

    print(f"""\n    
    homo_homo_match = {sim_homo_homo_match}
    homo_homo_1bp_diff = {sim_homo_homo_1bp_diff}
    homo_homo_40m_diff = {sim_homo_homo_40m_diff}
    homo_homo_mismatch = {sim_homo_homo_mismatch}
    
    homo_hetero_match_diff1bp = {sim_homo_hetero_match_diff1bp}
    homo_hetero_both_1bpdiff = {sim_homo_hetero_both_1bpdiff}
    homo_hetero_match_40m_diff = {sim_homo_hetero_match_40m_diff}
    homo_hetero_both_40m_diff = {sim_homo_hetero_both_40m_diff}
    homo_hetero_match_mismatch = {sim_homo_hetero_match_mismatch}
    homo_hetero_1bdfiff_40m_diff = {sim_homo_hetero_1bdfiff_40m_diff}
    homo_hetero_1bpdiff_mismatch = {sim_homo_hetero_1bpdiff_mismatch}
    homo_hetero_40m_diff_mismatch = {sim_homo_hetero_40m_diff_mismatch}
    homo_hetero_both_mismatch = {sim_homo_hetero_both_mismatch}""")
    
    print('\nHomozygous concordance = ', ((sim_homo_homo_match+sim_homo_homo_1bp_diff+sim_homo_hetero_match_diff1bp+sim_homo_hetero_both_1bpdiff)/common_loci_btw_HOMO_assembly_and_sim)*100)
    # the above categories are considered for final calculation of concordance


    
    sim_hetero_homo_one_match = 0
    sim_hetero_homo_1bp_diff = 0
    sim_hetero_homo_40m_diff = 0
    sim_hetero_homo_mismatch = 0
    
    sim_hetero_hetero_match_ = 0
    sim_hetero_hetero_match_diff1bp = 0
    sim_hetero_hetero_match_mismatch = 0
    sim_hetero_hetero_both_1bpdiff = 0
    sim_hetero_hetero_1bpdiff_mismatch = 0
    sim_hetero_hetero_both_mismatch = 0
    sim_hetero_hetero_match_40m_diff = 0
    sim_hetero_hetero_1bpdiff_40m_diff = 0
    sim_hetero_hetero_both_40m_diff = 0
    sim_hetero_hetero_40m_diff_mismatch = 0

    for locus_key in Assembly_heterozygous_dict:
    
        if locus_key in dicts:
            Value = dicts[locus_key]
            motif_thresh = 0.4*Assembly_motif_dict[locus_key]

            if Value[1] == 0: # homozygous
                
                sim_allele = Value[0]
                ass_allele = Assembly_heterozygous_dict[locus_key]
                if len(sim_allele & ass_allele) == 1:
                    sim_hetero_homo_one_match += 1
                        
                elif (abs(list(sim_allele)[0] - list(ass_allele)[0]) == 1) or (abs(list(sim_allele)[0] - list(ass_allele)[1]) == 1):
                    sim_hetero_homo_1bp_diff += 1
                            
                elif (abs(list(sim_allele)[0] - list(ass_allele)[0]) <= motif_thresh) or (abs(list(sim_allele)[0] - list(ass_allele)[1]) <= motif_thresh):
                    sim_hetero_homo_40m_diff += 1

                else:
                    sim_hetero_homo_mismatch += 1
                
            
            else:
                sim_allele = Value[0]
                ass_allele = Assembly_heterozygous_dict[locus_key]
                if sim_allele == ass_allele: 
                    sim_hetero_hetero_match_ += 1
                elif len(sim_allele & ass_allele) == 1:
                    diff = list(sim_allele ^ ass_allele)
                    if abs(diff[0] - diff[1]) == 1:
                       sim_hetero_hetero_match_diff1bp += 1
                    elif abs(diff[0] - diff[1]) <= motif_thresh:
                        sim_hetero_hetero_match_40m_diff += 1
                    else: sim_hetero_hetero_match_mismatch += 1
                else:
                    list_ass_allele = list(ass_allele)
                    list_sample_allele = list(sim_allele)
                    if ((abs(list_ass_allele[0] - list_sample_allele[0]) == 1) and (abs(list_ass_allele[1] - list_sample_allele[1]) == 1)) or ((abs(list_ass_allele[0] - list_sample_allele[1]) == 1) and (abs(list_ass_allele[1] - list_sample_allele[0]) == 1)):
                        sim_hetero_hetero_both_1bpdiff += 1
                    elif ((abs(list_ass_allele[0] - list_sample_allele[0]) == 1) or (abs(list_ass_allele[1] - list_sample_allele[1]) == 1)) or ((abs(list_ass_allele[0] - list_sample_allele[1]) == 1) or (abs(list_ass_allele[1] - list_sample_allele[0]) == 1)):
                        if (abs(list_ass_allele[0] - list_sample_allele[0]) == 1) or (abs(list_ass_allele[1] - list_sample_allele[1]) == 1):
                            if abs(list_ass_allele[0] - list_sample_allele[0]) == 1:
                                if abs(list_ass_allele[1] - list_sample_allele[1]) <= motif_thresh:
                                    sim_hetero_hetero_1bpdiff_40m_diff += 1
                                else:
                                    sim_hetero_hetero_1bpdiff_mismatch += 1
                            elif abs(list_ass_allele[0] - list_sample_allele[0]) <= motif_thresh:
                                sim_hetero_hetero_1bpdiff_40m_diff += 1
                            else:
                                sim_hetero_hetero_1bpdiff_mismatch += 1
                        elif abs(list_ass_allele[0] - list_sample_allele[1]) == 1:
                            if abs(list_ass_allele[1] - list_sample_allele[0]) <= motif_thresh:
                                sim_hetero_hetero_1bpdiff_40m_diff += 1
                            else:
                                sim_hetero_hetero_1bpdiff_mismatch += 1
                        elif abs(list_ass_allele[1] - list_sample_allele[0]) == 1:
                            if abs(list_ass_allele[0] - list_sample_allele[1]) <= motif_thresh:
                                sim_hetero_hetero_1bpdiff_40m_diff += 1
                            else:
                                sim_hetero_hetero_1bpdiff_mismatch += 1
                        else:
                            sim_hetero_hetero_1bpdiff_mismatch += 1
                            
                    elif ((abs(list_ass_allele[0] - list_sample_allele[0]) <= motif_thresh) and (abs(list_ass_allele[1] - list_sample_allele[1]) <= motif_thresh)) or ((abs(list_ass_allele[0] - list_sample_allele[1]) <= motif_thresh) and (abs(list_ass_allele[1] - list_sample_allele[0]) <= motif_thresh)):
                        sim_hetero_hetero_both_40m_diff += 1
                    elif ((abs(list_ass_allele[0] - list_sample_allele[0]) <= motif_thresh) or (abs(list_ass_allele[1] - list_sample_allele[1]) <= motif_thresh)) or ((abs(list_ass_allele[0] - list_sample_allele[1]) <= motif_thresh) or (abs(list_ass_allele[1] - list_sample_allele[0]) <= motif_thresh)):
                        sim_hetero_hetero_40m_diff_mismatch += 1
                    else: sim_hetero_hetero_both_mismatch += 1


    print(f'''\n    
    hetero_homo_one_match = {sim_hetero_homo_one_match}
    hetero_homo_1bp_diff = {sim_hetero_homo_1bp_diff}
    hetero_homo_40m_diff = {sim_hetero_homo_40m_diff}
    hetero_homo_mismatch = {sim_hetero_homo_mismatch}
    
    hetero_hetero_match_ = {sim_hetero_hetero_match_}
    hetero_hetero_match_diff1bp = {sim_hetero_hetero_match_diff1bp}
    hetero_hetero_match_40m_diff = {sim_hetero_hetero_match_40m_diff}
    hetero_hetero_match_mismatch = {sim_hetero_hetero_match_mismatch}
    hetero_hetero_both_1bpdiff = {sim_hetero_hetero_both_1bpdiff}
    hetero_hetero_1bpdiff_40m_diff = {sim_hetero_hetero_1bpdiff_40m_diff}
    hetero_hetero_1bpdiff_mismatch = {sim_hetero_hetero_1bpdiff_mismatch}
    hetero_hetero_both_40m_diff = {sim_hetero_hetero_both_40m_diff}
    hetero_hetero_40m_diff_mismatch = {sim_hetero_hetero_40m_diff_mismatch}
    hetero_hetero_both_mismatch = {sim_hetero_hetero_both_mismatch}\n\n\n''')
    
    print('\nHeterozygous concordance = ', ((sim_hetero_hetero_match_+sim_hetero_hetero_match_diff1bp+sim_hetero_hetero_both_1bpdiff)/common_loci_btw_HETERO_assembly_and_sim)*100)